In [1]:
"Hello"

'Hello'

In [2]:
# 테스트용 토픽 생성
from confluent_kafka.admin import AdminClient, NewTopic

admin = AdminClient({"bootstrap.servers": "kafka:9092"})

# 토픽 설정: 4개 파티션, 복제 1
topic = NewTopic(topic="api-events", num_partitions=4, replication_factor=1)

# 토픽 생성
fs = admin.create_topics([topic])

# 결과 확인
for topic_name, f in fs.items():
    try:
        f.result()  # 완료 대기
        print(f"토픽 '{topic_name}' 생성 완료!")
    except Exception as e:
        print(f"토픽 '{topic_name}' 생성 실패: {e}")

# 토픽 목록 확인
metadata = admin.list_topics(timeout=10)
print(
    f"\n현재 토픽 목록: {[t for t in metadata.topics.keys() if not t.startswith('_')]}"
)

토픽 'api-events' 생성 실패: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'api-events' already exists."}

현재 토픽 목록: ['api-events']


### Spark 연결 테스트

In [6]:
from pyspark.sql import SparkSession

In [ ]:
## SparkSession: Spark의 모든 기능을 사용하기 위한 진입점
spark =  SparkSession.builder.appName("ConnectionTest")\
    .master("spark://spark-master:7077")\
    .config("spark.executor.memory", "1g")\
    .config("spark.executor.cores", "1")\
    .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 02:12:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


26/01/19 02:13:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
#App ID:
spark.sparkContext.applicationId

'app-20260119021255-0000'

In [ ]:
# 간단한 테스트: 0~999까지 숫자 DataFrame 생성
df = spark.range(1000)
df

DataFrame[id: bigint]

In [28]:
df.columns

['id']

In [17]:
# 세션 종료 (리소스 반환)
spark.stop()

In [3]:
##Producer
from confluent_kafka import Producer
import json
import random
from datetime import datetime
import time

# -----------------------------------------------------------------------------
# Producer 설정
# -----------------------------------------------------------------------------
config = {
    "bootstrap.servers": "kafka:9092",
    "client.id": "api-event-producer",
}

producer = Producer(config)

# -----------------------------------------------------------------------------
# API 이벤트 생성 함수
# -----------------------------------------------------------------------------
ENDPOINTS = [
    "/api/products",
    "/api/users",
    "/api/orders",
    "/api/payments",
    "/api/search",
]
METHODS = ["GET", "POST", "PUT", "DELETE"]
STATUS_CODES = [200, 200, 200, 200, 201, 400, 404, 500]


def generate_api_event():
    """API 이벤트 데이터 생성"""
    return {
        "request_id": f"REQ_{random.randint(1, 999999):06d}",
        "user_id": f"U{random.randint(1, 1000):04d}",
        "endpoint": random.choice(ENDPOINTS),
        "method": random.choice(METHODS),
        "status_code": random.choice(STATUS_CODES),
        "response_time_ms": random.randint(10, 500),
        "timestamp": datetime.now().isoformat(),
    }


# -----------------------------------------------------------------------------
# Delivery Callback (전송 결과 확인)
# -----------------------------------------------------------------------------
sent_count = 0


def delivery_callback(err, msg):
    global sent_count
    if err:
        print(f"전송 실패: {err}")
    else:
        sent_count += 1


# -----------------------------------------------------------------------------
# 메시지 대량 전송
# -----------------------------------------------------------------------------
TOPIC = "api-events"
NUM_MESSAGES = 1000

print(f"API 이벤트 {NUM_MESSAGES}건 전송 시작...")
start_time = time.time()

for i in range(NUM_MESSAGES):
    event = generate_api_event()

    producer.produce(
        topic=TOPIC,
        value=json.dumps(event).encode("utf-8"),
        callback=delivery_callback,
    )

    # 1000건마다 진행 상황 출력
    if (i + 1) % 1000 == 0:
        producer.flush()
        print(f"  {i + 1}건 전송 완료")

producer.flush()
elapsed = time.time() - start_time

print("\n전송 완료!")
print(f"  총 전송: {sent_count}건")
print(f"  소요 시간: {elapsed:.2f}초")
print(f"  처리량: {sent_count / elapsed:.0f} records/sec")

API 이벤트 1000건 전송 시작...
  1000건 전송 완료

전송 완료!
  총 전송: 1000건
  소요 시간: 0.11초
  처리량: 8871 records/sec


In [4]:
# Step 2: 완성된 Consumer
from confluent_kafka import Consumer
import json
import time

# -----------------------------------------------------------------------------
# Consumer 설정
# -----------------------------------------------------------------------------
config = {
    "bootstrap.servers": "kafka:9092",
    "group.id": "api-event-consumer-group",
    "auto.offset.reset": "earliest",  # 처음부터 읽기
    "enable.auto.commit": True,
}

consumer = Consumer(config)
consumer.subscribe(["api-events"])

# -----------------------------------------------------------------------------
# 메시지 수신 및 통계
# -----------------------------------------------------------------------------
print("메시지 수신 시작...\n")

stats = {"total": 0, "by_status": {}, "by_endpoint": {}}
start_time = time.time()
max_messages = 100  # 100개 메시지 수신 후 통계 출력

try:
    while stats["total"] < max_messages:
        msg = consumer.poll(timeout=1.0)

        if msg is None:
            continue
        if msg.error():
            print(f"에러: {msg.error()}")
            continue

        # 메시지 파싱
        event = json.loads(msg.value().decode("utf-8"))

        # 통계 업데이트
        stats["total"] += 1

        status = event["status_code"]
        stats["by_status"][status] = stats["by_status"].get(status, 0) + 1

        endpoint = event["endpoint"]
        stats["by_endpoint"][endpoint] = stats["by_endpoint"].get(endpoint, 0) + 1

        # 10개마다 진행 상황
        if stats["total"] % 100 == 0:
            print(f"  수신: {stats['total']}건")

finally:
    consumer.close()

# -----------------------------------------------------------------------------
# 통계 출력
# -----------------------------------------------------------------------------
elapsed = time.time() - start_time

print(f"\n{'=' * 50}")
print(f"총 수신: {stats['total']}건 ({elapsed:.2f}초)")
print(f"처리량: {stats['total'] / elapsed:.0f} records/sec")

print(f"\n상태 코드별 분포:")
for status, count in sorted(stats["by_status"].items()):
    pct = count / stats["total"] * 100
    print(f"  {status}: {count}건 ({pct:.1f}%)")

print(f"\n엔드포인트별 분포:")
for endpoint, count in sorted(stats["by_endpoint"].items(), key=lambda x: -x[1]):
    pct = count / stats["total"] * 100
    print(f"  {endpoint}: {count}건 ({pct:.1f}%)")

메시지 수신 시작...

  수신: 100건

총 수신: 100건 (3.09초)
처리량: 32 records/sec

상태 코드별 분포:
  200: 60건 (60.0%)
  201: 5건 (5.0%)
  400: 9건 (9.0%)
  404: 10건 (10.0%)
  500: 16건 (16.0%)

엔드포인트별 분포:
  /api/users: 27건 (27.0%)
  /api/search: 21건 (21.0%)
  /api/payments: 18건 (18.0%)
  /api/products: 18건 (18.0%)
  /api/orders: 16건 (16.0%)


In [1]:
# Consumer: 병렬 처리 (노트북 2개에서 동시 실행)
from confluent_kafka import Consumer
import json
import time

config = {
    "bootstrap.servers": "kafka:9092",
    "group.id": "parallel-consumer-group",  # 같은 그룹 ID!
    "auto.offset.reset": "earliest",
}

consumer = Consumer(config)
consumer.subscribe(["api-events"])

print("Consumer 시작...")
count = 0
start = None
last_msg_time = None

empty_count = 0
max_empty = 3  # 1초 x 3번 = 3초 대기 후 종료
first_message = True

try:
    while True:
        # 첫 메시지는 10초 대기, 이후 1초 대기
        timeout = 10.0 if first_message else 1.0
        msg = consumer.poll(timeout)

        if msg is None:
            empty_count += 1
            if first_message:
                print("10초 동안 메시지 없음. 종료합니다.")
                break
            if empty_count >= max_empty:
                print("더 이상 메시지 없음. 종료합니다.")
                break
            continue

        if msg.error():
            continue

        empty_count = 0

        # 첫 메시지 수신 시 시간 측정 시작
        if first_message:
            start = time.time()
            first_message = False

        last_msg_time = time.time()
        count += 1

        if count % 100000 == 0:
            print(f"  {count:,}건 처리중 (파티션 {msg.partition()})")

except KeyboardInterrupt:
    pass
finally:
    consumer.close()

# 결과 출력 (대기 시간 제외, 실제 처리 시간만 측정)
print(f"\n{'=' * 50}")
if count > 0 and start and last_msg_time:
    elapsed = last_msg_time - start
    print(f"총 수신: {count:,}건")
    print(f"소요 시간: {elapsed:.2f}초 (대기 시간 제외)")
    print(f"처리량: {count / elapsed:,.0f} records/sec")
else:
    print("수신된 메시지 없음")

Consumer 시작...
  100,000건 처리중 (파티션 2)
  200,000건 처리중 (파티션 2)
  300,000건 처리중 (파티션 2)
  400,000건 처리중 (파티션 2)
더 이상 메시지 없음. 종료합니다.

총 수신: 491,979건
소요 시간: 2.43초 (대기 시간 제외)
처리량: 202,152 records/sec
